In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# nltk.download('all')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/farzi-test/Test_DJIA_Table.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/sample_submission.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_DJIA_Table.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/TEST_Redit_news.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/RedditNews(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_Combined_News.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
news = pd.read_csv("../input/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv")
df_reddit_news = pd.read_csv("../input/stock-market-prediction-and-sentimental-analysis/RedditNews(train).csv")
df = pd.read_csv("../input/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from collections import Counter

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df.shape

/tmp/ipykernel_18/1056846339.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


(1863, 7)

In [5]:
news['Date'] = pd.to_datetime(news['Date'])
news.shape

(1989, 27)

In [6]:
news = news[df['Date'].max() >= news['Date']]
news = news[news['Date'] >= df['Date'].min()]
news.shape

(1863, 27)

In [7]:
news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [8]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-31,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930
1,2015-12-30,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914
2,2015-12-29,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047
3,2015-12-28,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953
4,2015-12-24,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992


In [9]:
news.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1858,2015-12-24,0,"Brazil declares emergency after 2,400 babies a...",Scientists find big 1500-yr-old Viking settlem...,Paris attacks: Belgian police arrest ninth sus...,Wiretapping reveals communication between Turk...,Russia airstrikes: Hundreds of Syrian civilans...,Pakistan changes neutral position on Syrian Ci...,North Korean diplomat arrested in South Africa...,German teachers want 'Mein Kampf' on syllabus,...,Report on sexual exploitation and abuse by pea...,Baby girl dies after X-president's son's staff...,China smog sparks red alerts in 10 cities,US planned East Berlin's 'systematic destructi...,Qatar World Cup 2022 workers 'earning as littl...,Thai Court Sentences Two to Death for Killings...,Germany Pays to Halt Danish Wind Power to Prot...,Egypts President Sisi Urges Islamic Scholars t...,Russia Rejects Amnesty Internationals Report o...,"Cameron, Osborne and Murdoch back together at ..."
1859,2015-12-28,0,"Germany recruits 8,500 teachers to teach Germa...","Almost 23,000 people have signed an Italian po...",Japan apologizes to Korea over sexual enslavem...,Milan and Rome ban cars as pollution rises. Mi...,Saudi posts record $98 bn deficit in 2015,"The United States, Russia, Iran, and Saudi Ara...",Vladimir Putin has achieved his central goal o...,Noted Syrian journalist who uncovered ISIS Ale...,...,Saudi Arabia Plans Subsidy Cuts as King Unveil...,Syria anti-ISIS documentary maker 'assassinate...,Denmark wants Geneva Convention debate if Euro...,David Cameron is heckled during visit to flood...,British Army Is Deployed as Flooding Submerges...,Brazils reluctance to accept an Israeli ambass...,Russian GDP down 4.0 percent year-on-year in N...,Syria and Iraq: Ethnic cleansing by Sunni and ...,Deadly car bomb explodes at Kabul airport,Experiment in Germany tests the idea of an 'em...
1860,2015-12-29,1,Pentagon thwarts Obama's effort to close Guant...,Rare Footage of Giant Squid in Japan,Muslims tell ISIS they'd rather see Star Wars ...,Syrian Journalist Who Documented ISIS Atrociti...,North Korea's second most-powerful man sent to...,Austria Turns Away Hundreds of Migrants for Ly...,Saudi Grand Mufti calls ISIS part of the Israe...,"'What has war yielded, neither land nor heaven...",...,Iran Hands Over Stockpile of Enriched Uranium ...,Registrar Shuts Down All Pirate Bay Domain Names,The Irish Government has taken unprecedented a...,Saudi Arabia to raise domestic petrol prices b...,Saudi Arabia hikes petrol prices by 40% at the...,World Health Organization declares Guinea Ebol...,Hacker group Anonymous is helping the 'Making ...,Russia 'has removed uranium from Iran',China threatens to shoot down Australian aircr...,Israel warns Brazil to accept its settler amba...
1861,2015-12-30,0,Tech companies face criminal charges if they n...,Mexican marijuana farmers see profits tumble a...,A would-be suicide bomber and his wife have be...,The rich will get richer while two million mor...,The North Pole will have above freezing temper...,South Australia - New sexting laws to spare ch...,ISIS militants burn 20 media activists to deat...,Italy fines Apple over misleading tax conduct,...,Norway to turn back asylum seekers on Swedish ...,Islamic State Suffers Double Blow As Ramadi Fa...,Journalist reveals Erdoan imprisoned him for e...,Russian Moon base plan falls victim to budget ...,"Belgium, Netherlands to exchange territory wi...",North Korea says top official Kim Yang-gon kil...,Putin's Son-in-Law Boosted by $1.75 Bln Russia...,An orgasm a day could cut prostate cancer risk,"Germans claim Belgian nuclear reactors are ""fa...",UK Sperm Bank Doesn't Accept Donors with Dysle...
1862,2015-12-31,0,Microsoft Corp said on Wednesday it will begin...,Greek Orthodox Church: Homosexuals are God's c...,"Vladmir Putin officially dissolves Roscosmos, ...",Donald Trump could be refused entry to the UK ...,Saudi Arabia says it

In [10]:
ps = PorterStemmer()
wordnet = WordNetLemmatizer()

In [11]:
data = news.iloc[:, 2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...,b Russian tanks are moving towards the capital...,b Afghan children raped with impunity U N ...,b Russian tanks have entered South Ossetia...,b Breaking Georgia invades South Ossetia Rus...,b The enemy combatent trials are nothing but...,b Georgian troops retreat from S Osettain cap...,b Did the U S Prep Georgia for War with Russia,...,b Georgia Invades South Ossetia if Russia ge...,b Al Qaeda Faces Islamist Backlash,b Condoleezza Rice The US would not act to p...,b This is a busy day The European Union has ...,b Georgia will withdraw soldiers from Ir...,b Why the Pentagon Thinks Attacking Iran is a ...,b Caucasus in crisis Georgia invades South Os...,b Indian shoe manufactory And again in a se...,b Visitors Suffering from Mental Illnesses Ban...,b No Help for Mexico s Kidnapping Surge
1,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...,b Georgian army flees in disarray as Russians ...,b Olympic opening ceremony fireworks faked,b What were the Mossad with fraudulent New Zea...,b Russia angered by Israeli military sale to G...,b An American citizen living in S Ossetia blam...,b Welcome To World War IV Now In High Definit...,b Georgia s move a mistake of monumental prop...,...,b Israel and the US behind the Georgian aggres...,b Do not believe TV neither Russian nor Geor...,b Riots are still going on in Montreal Canada...,b China to overtake US as largest manufacturer,b War in South Ossetia PICS,b Israeli Physicians Group Condemns State Tort...,b Russia has just beaten the United States ov...,b Perhaps the question about the Georgia R...,b Russia is so much better at war,b So this is what it s come to trading sex fo...
2,b Remember that adorable year old who sang a...,b Russia ends Georgia operation,b If we had no sexual harassment we would hav...,b Al Qa eda is losing support in Iraq because ...,b Ceasefire in Georgia Putin Outmaneuvers the...,b Why Microsoft and Intel tried to kill the XO...,b Stratfor The Russo Georgian War and the Bal...,b I m Trying to Get a Sense of This Whole Geor...,b The US military was surprised by the timing ...,b U S Beats War Drum as Iran Dumps the Dollar,...,b U S troops still in Georgia did you know t...,b Why Russias response to Georgia was right,b Gorbachev accuses U S of making a serious ...,b Russia Georgia and NATO Cold War Two,b Remember that adorable year old who led y...,b War in Georgia The Israeli connection,b All signs point to the US encouraging Georgi...,b Christopher King argues that the US and NATO...,b America The New Mexico,b BBC NEWS Asia Pacific Extinction by man...
3,b U S refuses Israel weapons to attack Iran ...,b When the president ordered to attack Tskhinv...,b Israel clears troops who killed Reuters cam...,b Britain s policy of being tough on drugs is...,b Body of year old found in trunk Latest ...,b China has moved million quake survivors...,b Bush announces Operation Get All Up In Russi...,b Russian forces sink Georgian ships,b The commander of a Navy air reconnaissance s...,b of CNN readers Russia s actions in Geor...,...,b Elephants extinct by,b US humanitarian missions soon in Georgia i...,b Georgia s DDOS came from US sources,b Russian convoy heads into Georgia violating...,b Israeli defence minister US against strike ...,b Gorbachev We Had No Choice,b Witness Russian forces head towards Tbilisi...,b Quarter of Russians blame U S for conflict...,b Georgian president says US military will ta...,b Nobel laureate Aleksander Solzhenitsyn...
4,b All the experts admit that we should legalis...,b War in South Osetia pictures made by a ...,b Swedish wrestler Ara Abrahamian throws away ...,b Russia exaggerated the death toll in South O...,b Missile That Killed Inside Pakistan May Ha...,b Rushdie Cond

In [12]:
analyzer = SentimentIntensityAnalyzer()
headlines = []
for row in range(0,len(data.index)):
    headlines.append( ' '.join(str(x) for x in data.iloc[row,0:25]))

In [13]:
stop_words = stopwords.words('english')
stop_words_dict = Counter(stop_words)
for i in range(data.shape[0]):
    headlines[i] = ' '.join([ps.stem(word) for word in headlines[i].split() if not word in stop_words_dict and word != 'b'])

In [14]:
headlines[0]

'georgia down two russian warplan countri move brink war break musharraf impeach russia today column troop roll south ossetia footag fight youtub russian tank move toward capit south ossetia reportedli complet destroy georgian artilleri fire afghan children rape impun U N offici say sick three year old rape noth russian tank enter south ossetia whilst georgia shoot two russian jet break georgia invad south ossetia russia warn would interven SO side the enemi combat trial noth sham salim haman sentenc year kept longer anyway feel like georgian troop retreat S osettain capit presum leav sever hundr peopl kill video did U S prep georgia war russia rice give green light israel attack iran say U S veto isra militari op announc class action lawsuit behalf american public against fbi So russia georgia war nyt top stori open ceremoni olymp what fuck disgrac yet proof declin journal china tell bush stay countri affair did world war iii start today georgia invad south ossetia russia get involv n

In [15]:
scores = []
for i in range(len(headlines)):
    a = (analyzer.polarity_scores(headlines[i]))
    scores.append([a['neg'], a['neu'], a['pos']])

In [16]:
scores[0:4]

[[0.265, 0.688, 0.047],
 [0.247, 0.69, 0.063],
 [0.212, 0.755, 0.033],
 [0.153, 0.787, 0.06]]

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [18]:
# countvector=CountVectorizer(ngram_range=(2,2))
countvector=TfidfVectorizer(ngram_range=(1,1))

In [19]:
traindataset=countvector.fit_transform(headlines)

In [20]:
traindataset = np.array(traindataset.todense())
traindataset.shape

(1863, 21178)

In [21]:
randomclassifier=RandomForestClassifier()
randomclassifier.fit(traindataset,news['Label'])

RandomForestClassifier()

In [22]:
predictions = randomclassifier.predict_proba(traindataset)

In [23]:
predictions

array([[0.78, 0.22],
       [0.13, 0.87],
       [0.85, 0.15],
       ...,
       [0.18, 0.82],
       [0.89, 0.11],
       [0.83, 0.17]])

In [24]:
predict1 = predictions[:, 1]

In [25]:
open_price = np.array(df['Open'])
high_price = np.array(df['High'])

In [26]:
     low_price = np.array(df['Low'])

In [27]:
train = np.stack((predict1, open_price, low_price, high_price), axis=1) 

In [28]:
train.shape

(1863, 4)

In [29]:
scores = np.array(scores)
scores.shape
train = np.append(scores, train, axis = 1)


In [30]:
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor

In [31]:
trainY = df['Close']

In [32]:
regr = LinearRegression()
rfreg=RandomForestRegressor()
rfreg.fit(train, trainY)
regr.fit(train, trainY)
print(regr.score(train, trainY))

0.9997000373415479


In [33]:

test_news=pd.read_csv('../input/stock-market-prediction-and-sentimental-analysis/Test_Combined_News.csv')

In [34]:
test_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,04-01-2016,0,Saudi Arabia breaks off ties with Iran - Amid ...,Exclusive: UK Government urged to reveal its r...,China halts trading as stocks fall 7%,Earthquake measuring 6.8 magnitude strikes India,A virus linked to shrinking newborns brains is...,"New photos of SpaceX booster show sooty, but u...",Indian consulate in Afghanistan under attack b...,Iran calls off Major Hajj,...,Iranian leaders website posts controversial im...,"After Bahrain, Sudan expels Iranian ambassador...",Saudi Arabia to cut off all air traffic with I...,Hezbollah sets off large border bomb targeting...,Iranian diplomats given 48 hours to leave Saud...,BBC News: Migrant crisis: Sweden border checks...,Teachers claim the politics of austerity is le...,The Aftermath of a Muslim Cleric's Execution i...,Iran: 40 arrested for attack on Saudi embassy,"Oil prices jump over 2 pct on Saudi Arabia, Ir..."
1,05-01-2016,1,"Dutch government backs strong encryption, cond...",ISIS Militants Strap Bomb on 4 Year-Old Boy an...,Canada proceeding with controversial $15-billi...,Brazil is heading for its worst recession sinc...,British ISIS militant's desperate plea for doc...,Malaysia's schools will start holding anti-ISI...,Saudi Arabia omitted from UK death penalty str...,ISIS executes first female journalist. One of ...,...,'Fat cat Tuesday' as top bosses pay overtakes ...,"""The United Nations special rapporteur for hum...",Merkel rejects Bavarian bid for refugee limit:...,Yemen centre for blind 'hit in Saudi coalition...,Mass murderer Anders Behring Breiviks court ca...,Kuwait has recalled its ambassador to Iran in ...,Britain set for first mass strike by doctors i...,First known hacker-caused power outage signals...,Kuwait becomes latest Saudi ally to downgrade ...,'Do Your Cars Emit Oxygen?' Indian Supreme Cou...
2,06-01-2016,0,China shuts down stock market after 15 min 7% ...,Armed suspect shot dead after trying to storm ...,Oil price falls below $35 a barrel to fresh 11...,Germans outraged by mayors advice for women af...,German public broadcaster ZDF has apologised f...,Iran accuses Saudis of bombing their embassy i...,Protesters demand Canada cancel $15B Saudi con...,"'Snooper's charter' will cost British lives, M...",...,Rio Olympic stadium has no water or electricit...,Saudi-led air strikes in Yemen hit centre for ...,Truck bomb kills 65 at Libyan police training ...,Israeli Tourists on Tour Bus Attacked by Maske...,Israel and China have signed a document of und...,Donald Trump threatens to pull 700m investment...,Israel busts Hamas terror cell planning abduct...,A working hospital in central China was demoli...,Paris to close Champs Elysees to cars one Sund...,TransCanada Corp files US$15-billion lawsuit a...
3,07-01-2016,0,Reports of sexual assaults on women across Eur...,Shelter forcibly converted children to Christi...,Iran Has Banned Its Citizens from Making Pilgr...,Gaddafi warned Blair his ousting would 'open d...,A Turkish firm has been selling fake life vest...,German minister says deportations possible in ...,Mao statue in China 'pulled down by authoritie...,A poacher-hunting dog in South Africa has been...,...,Charlie Hebdo anniversary: free-speech groups ...,Polish president signs bill putting state medi...,Mein Kampf hits stores in tense Germany,"Morocco Bans WhatsApp, Skype, Viber and Other ...",The UN library announced its most-checked-out ...,One third of the worlds freshwater fish are at...,"Nuclear weapons risk greater than in cold war,...","North Korea Claims That It Tested the ""H-Bomb ...",UK - DWP told woman she was not ill enough for...,Venezuela's new parliament removes pictures of...
4,08-01-2016,0,El Chapo' Being Taken to Same Prison He Escape...,El Chapo has been caught,At least 231 children abused at Catholic boys ...,North Korea has defended its latest nuclear te...,A group of United Nations' experts have warned...

In [35]:
test_news.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
121,27-06-2016,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
122,28-06-2016,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
123,29-06-2016,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
124,30-06-2016,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
125,01-07-2016,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates urge 

In [36]:
test_data = test_news.iloc[:, 2:27]
test_data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
test_list= [i for i in range(25)]
test_new_Index=[str(i) for i in test_list]
test_data.columns= test_new_Index
test_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,Saudi Arabia breaks off ties with Iran Amid ...,Exclusive UK Government urged to reveal its r...,China halts trading as stocks fall,Earthquake measuring magnitude strikes India,A virus linked to shrinking newborns brains is...,New photos of SpaceX booster show sooty but u...,Indian consulate in Afghanistan under attack b...,Iran calls off Major Hajj,Denmark responds to Swedish border checks with...,David Cameron criticised for turning blind ey...,...,Iranian leaders website posts controversial im...,After Bahrain Sudan expels Iranian ambassador...,Saudi Arabia to cut off all air traffic with I...,Hezbollah sets off large border bomb targeting...,Iranian diplomats given hours to leave Saud...,BBC News Migrant crisis Sweden border checks...,Teachers claim the politics of austerity is le...,The Aftermath of a Muslim Cleric s Execution i...,Iran arrested for attack on Saudi embassy,Oil prices jump over pct on Saudi Arabia Ir...
1,Dutch government backs strong encryption cond...,ISIS Militants Strap Bomb on Year Old Boy an...,Canada proceeding with controversial billi...,Brazil is heading for its worst recession sinc...,British ISIS militant s desperate plea for doc...,Malaysia s schools will start holding anti ISI...,Saudi Arabia omitted from UK death penalty str...,ISIS executes first female journalist One of ...,The U S Justice Department has sued Volkswage...,Leaked memo shows Saudi Arabia was fully prepa...,...,Fat cat Tuesday as top bosses pay overtakes ...,The United Nations special rapporteur for hum...,Merkel rejects Bavarian bid for refugee limit ...,Yemen centre for blind hit in Saudi coalition...,Mass murderer Anders Behring Breiviks court ca...,Kuwait has recalled its ambassador to Iran in ...,Britain set for first mass strike by doctors i...,First known hacker caused power outage signals...,Kuwait becomes latest Saudi ally to downgrade ...,Do Your Cars Emit Oxygen Indian Supreme Cou...
2,China shuts down stock market after min ...,Armed suspect shot dead after trying to storm ...,Oil price falls below a barrel to fresh ...,Germans outraged by mayors advice for women af...,German public broadcaster ZDF has apologised f...,Iran accuses Saudis of bombing their embassy i...,Protesters demand Canada cancel B Saudi con...,Snooper s charter will cost British lives M...,Islamic scholars from Malaysia and Indonesia i...,Turkish workshop employs Syrian minors to prod...,...,Rio Olympic stadium has no water or electricit...,Saudi led air strikes in Yemen hit centre for ...,Truck bomb kills at Libyan police training ...,Israeli Tourists on Tour Bus Attacked by Maske...,Israel and China have signed a document of und...,Donald Trump threatens to pull m investment...,Israel busts Hamas terror cell planning abduct...,A working hospital in central China was demoli...,Paris to close Champs Elysees to cars one Sund...,TransCanada Corp files US billion lawsuit a...
3,Reports of sexual assaults on women across Eur...,Shelter forcibly converted children to Christi...,Iran Has Banned Its Citizens from Making Pilgr...,Gaddafi warned Blair his ousting would open d...,A Turkish firm has been selling fake life vest...,German minister says deportations possible in ...,Mao statue in China pulled down by authoritie...,A poacher hunting dog in South Africa has been...,Police used terror powers to spy on officers ...,Iran Bans All Imports from Saudi Arabia,...,Charlie Hebdo anniversary free speech groups ...,Polish president signs bill putting state medi...,Mein Kampf hits stores in tense Germany,Morocco Bans WhatsApp Skype Viber and Other ...,The UN library announced its most checked out ...,One third of the worlds freshwater fish are at...,Nuclear weapons risk greater than in cold war ...,North Korea Claims That It Tested the H Bomb ...,UK DWP told woman she was not ill enough for...,Venezuela s new parliament removes pictures of...
4,El Chapo Being Taken to Same Prison He Escape...,El Chapo

In [37]:
test_headlines = []
for row in range(0,len(test_data.index)):
    test_headlines.append(' '.join(str(x) for x in test_data.iloc[row,0:25]))

In [38]:
for i in range(test_data.shape[0]):
    test_headlines[i] = ' '.join([ps.stem(word) for word in test_headlines[i].split() if not word in stop_words_dict])

In [39]:

test_headlines.reverse()
test_dataset=countvector.transform(test_headlines)
test_d = pd.DataFrame(test_dataset.todense())
test_scores = []
for i in range(len(test_headlines)):
    test_a = (analyzer.polarity_scores(test_headlines[i]))
    test_scores.append([test_a['neg'], test_a['neu'], test_a['pos']])

In [40]:
predictions = randomclassifier.predict_proba(test_dataset)

In [41]:
predict2 = predictions[:, 1]

In [42]:
test_TABLE=pd.read_csv('../input/stock-market-prediction-and-sentimental-analysis/Test_DJIA_Table.csv')


In [43]:
test_TABLE.head()

,Date,Open,High,Low,Volume
0,01-07-2016,17924.24023,18002.38086,17916.91016,82160000
1,30-06-2016,17712.75977,17930.60938,17711.80078,133030000
2,29-06-2016,17456.01953,17704.50977,17456.01953,106380000
3,28-06-2016,17190.50977,17409.72070,17190.50977,112190000
4,27-06-2016,17355.21094,17355.21094,17063.08008,138740000


In [44]:
test_open=test_TABLE['Open']
test_high=test_TABLE['High']
test_low=test_TABLE['Low']

In [45]:
test = np.stack((predict2, test_open, test_high, test_low), axis=1) 

In [46]:
test_scores = np.array(test_scores)
test_scores.shape
test = np.append(test_scores, test, axis = 1)

In [47]:
ans = regr.predict(test)*100000
ans2 = rfreg.predict(test)*100000

In [48]:
index = [i + 1 for i in range(len(ans))]

In [49]:
dict = {'ID': index, 'Close': ans}
dict2 = {'ID': index, 'Close': ans2}
     
df = pd.DataFrame(dict)
df2 = pd.DataFrame(dict2)
# saving the dataframe
df.head(5)

,ID,Close
0,1,1.798015e+09
1,2,1.788359e+09
2,3,1.765127e+09
3,4,1.736080e+09
4,5,1.710218e+09


In [50]:
df2.head()

,ID,Close
0,1,1.798727e+09
1,2,1.781252e+09
2,3,1.762058e+09
3,4,1.732852e+09
4,5,1.729680e+09


In [51]:
ans2

array([1.79872725e+09, 1.78125162e+09, 1.76205762e+09, 1.73285174e+09,
       1.72968009e+09, 1.77116705e+09, 1.79667768e+09, 1.78296813e+09,
       1.78061972e+09, 1.78285433e+09, 1.77054985e+09, 1.76889028e+09,
       1.77219157e+09, 1.77092484e+09, 1.77850716e+09, 1.78493665e+09,
       1.79856204e+09, 1.79946634e+09, 1.79982965e+09, 1.78597042e+09,
       1.77610385e+09, 1.77444271e+09, 1.77283107e+09, 1.77887508e+09,
       1.78126279e+09, 1.78011856e+09, 1.77857927e+09, 1.76950509e+09,
       1.75639937e+09, 1.75851434e+09, 1.73884322e+09, 1.75964453e+09,
       1.76278400e+09, 1.76950348e+09, 1.76986284e+09, 1.77365145e+09,
       1.78107439e+09, 1.78171039e+09, 1.77483420e+09, 1.76850662e+09,
       1.77126471e+09, 1.77183993e+09, 1.77576649e+09, 1.78351674e+09,
       1.77395133e+09, 1.78429322e+09, 1.80125385e+09, 1.80029366e+09,
       1.79691788e+09, 1.79901924e+09, 1.80170998e+09, 1.80391995e+09,
       1.80348490e+09, 1.79615849e+09, 1.79578686e+09, 1.79810169e+09,
      

In [52]:
ans

array([1.79801509e+09, 1.78835942e+09, 1.76512709e+09, 1.73608017e+09,
       1.71021787e+09, 1.74424118e+09, 1.79746393e+09, 1.78505747e+09,
       1.78449338e+09, 1.79011650e+09, 1.76237843e+09, 1.76091651e+09,
       1.76856581e+09, 1.76310883e+09, 1.77973652e+09, 1.78304135e+09,
       1.79515258e+09, 1.79977456e+09, 1.79878768e+09, 1.79202420e+09,
       1.77343321e+09, 1.77551779e+09, 1.77206771e+09, 1.77586646e+09,
       1.78600221e+09, 1.78300869e+09, 1.78571858e+09, 1.76937172e+09,
       1.75191265e+09, 1.75423288e+09, 1.73586937e+09, 1.75391068e+09,
       1.75055075e+09, 1.77061004e+09, 1.75592192e+09, 1.77050220e+09,
       1.77407105e+09, 1.78896610e+09, 1.77098149e+09, 1.76632144e+09,
       1.76789575e+09, 1.76313555e+09, 1.77016188e+09, 1.78748764e+09,
       1.76751778e+09, 1.78394782e+09, 1.79991108e+09, 1.79866208e+09,
       1.78748834e+09, 1.79551280e+09, 1.79895734e+09, 1.81205022e+09,
       1.80592025e+09, 1.79480373e+09, 1.78829518e+09, 1.79264455e+09,
      

In [53]:
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor()  

In [54]:
regressor.fit(train, trainY)

DecisionTreeRegressor()

In [55]:
ans3 = regressor.predict(test)

In [56]:
ans3

array([17983.07031, 17678.69922, 17613.74023, 17122.00977, 17122.00977,
       17687.82031, 18057.65039, 17729.21094, 17729.21094, 17635.39063,
       17568.5293 , 17687.82031, 17687.82031, 17678.69922, 17719.     ,
       17730.10938, 18024.16992, 18038.23047, 18024.16992, 17732.75   ,
       17687.82031, 17635.39063, 17635.39063, 17719.     , 17730.10938,
       17732.75   , 17635.39063, 17687.82031, 17613.74023, 17619.50977,
       16990.68945, 17712.66016, 17687.82031, 17687.82031, 17687.82031,
       17687.82031, 17687.82031, 17635.39063, 17687.82031, 17687.82031,
       17635.39063, 17719.     , 17678.23047, 17764.03906, 17687.82031,
       17729.21094, 18057.65039, 17983.07031, 18057.65039, 17983.07031,
       18080.14063, 17983.07031, 18057.65039, 17983.07031, 18038.26953,
       18057.65039, 17635.39063, 17687.82031, 17687.82031, 17687.82031,
       17635.39063, 17687.82031, 17687.82031, 17687.82031, 17687.82031,
       17687.82031, 17719.     , 17623.05078, 17619.50977, 17477

In [57]:
ans3 = np.array(ans3)*100000

In [58]:
ans3

array([1.79830703e+09, 1.76786992e+09, 1.76137402e+09, 1.71220098e+09,
       1.71220098e+09, 1.76878203e+09, 1.80576504e+09, 1.77292109e+09,
       1.77292109e+09, 1.76353906e+09, 1.75685293e+09, 1.76878203e+09,
       1.76878203e+09, 1.76786992e+09, 1.77190000e+09, 1.77301094e+09,
       1.80241699e+09, 1.80382305e+09, 1.80241699e+09, 1.77327500e+09,
       1.76878203e+09, 1.76353906e+09, 1.76353906e+09, 1.77190000e+09,
       1.77301094e+09, 1.77327500e+09, 1.76353906e+09, 1.76878203e+09,
       1.76137402e+09, 1.76195098e+09, 1.69906895e+09, 1.77126602e+09,
       1.76878203e+09, 1.76878203e+09, 1.76878203e+09, 1.76878203e+09,
       1.76878203e+09, 1.76353906e+09, 1.76878203e+09, 1.76878203e+09,
       1.76353906e+09, 1.77190000e+09, 1.76782305e+09, 1.77640391e+09,
       1.76878203e+09, 1.77292109e+09, 1.80576504e+09, 1.79830703e+09,
       1.80576504e+09, 1.79830703e+09, 1.80801406e+09, 1.79830703e+09,
       1.80576504e+09, 1.79830703e+09, 1.80382695e+09, 1.80576504e+09,
      